In [ ]:
import csv
import json
import requests

def csv_to_json(url, json_file_path):
    try:
        # Step 1: Fetch CSV data from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        csv_data = response.text

        # Step 2: Parse the CSV data
        csv_reader = csv.DictReader(csv_data.splitlines())
        data_list = []

        # Step 3: Process rows and calculate clanscore
        for row in csv_reader:
            clanscore = 0
            # Add all numeric values in the row to clanscore
            for value in row.values():
                try:
                    # Attempt to convert the value to a int and add it to clanscore
                    clanscore += int(value)
                except ValueError:
                    # Skip non-numeric values (e.g., text fields)
                    continue
            # Convert clanscore to string and add it as a new column
            row['clanscore'] = str(clanscore)
            data_list.append(row)

        # Step 4: Save data as JSON with UTF-8 encoding and ensure ASCII is False to handle special characters
        with open(json_file_path, 'w', encoding='utf-8') as json_file:
            json.dump(data_list, json_file, indent=4, ensure_ascii=False)

        print(f"CSV data successfully converted to JSON with clanscore and saved to {json_file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching CSV: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

csv_url = "https://raw.githubusercontent.com/Lightning-President-9/ClanDataRepo/refs/heads/main/Clan%20Members/Monthly%20Analysis/data_APR-MAY_2025.csv"
output_json_path = "data_APR-MAY_2025.json"
csv_to_json(csv_url, output_json_path)

CSV data successfully converted to JSON with clanscore and saved to data_APR-MAY_2025.json


In [ ]:
import requests
import pandas as pd
from collections import defaultdict

# List of (url, month label) tuples in chronological order
monthly_files = [
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_JUL-AUG_2024.json', 'JUL-AUG_2024'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_AUG-SEP_2024.json', 'AUG-SEP_2024'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_SEP-OCT_2024.json', 'SEP-OCT_2024'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_OCT-NOV_2024.json', 'OCT-NOV_2024'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_NOV-DEC_2024.json', 'NOV-DEC_2024'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_DEC-JAN_2025.json', 'DEC-JAN_2025'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_JAN-FEB_2025.json', 'JAN-FEB_2025'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_FEB-MAR_2025.json', 'FEB-MAR_2025'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_MAR-APR_2025.json', 'MAR-APR_2025'),
    ('https://github.com/Lightning-President-9/ClanDataRepo/raw/main/Clan%20Members/Monthly%20Analysis%20JSON/data_APR-MAY_2025.json', 'APR-MAY_2025')
]

# Collect all player data
all_data = defaultdict(dict)

for url, label in monthly_files:
    print(f"Processing {label}...")
    response = requests.get(url)
    response.raise_for_status()
    month_data = response.json()

    for player in month_data:
        name = player['name'].strip()
        all_data[name][f'warattack_{label}'] = int(player['warattack'])
        all_data[name][f'clancapital_{label}'] = int(player['clancapital'])
        all_data[name][f'clangames_{label}'] = int(player['clangames'])
        all_data[name][f'clangamesmaxed_{label}'] = int(player['clangamesmaxed'])
        all_data[name][f'clanscore_{label}'] = int(player['clanscore'])

# Ensure consistent column order: first by month, then by metric
metrics = ['warattack', 'clancapital', 'clangames', 'clangamesmaxed', 'clanscore']
ordered_columns = [f"{metric}_{label}" for label in [label for _, label in monthly_files] for metric in metrics]

# Add 'name' as the first column
columns = ['name'] + ordered_columns

# Create rows
rows = []
for name, stats in all_data.items():
    row = {'name': name}
    for col in ordered_columns:
        row[col] = stats.get(col, -1)
    rows.append(row)

# Save as DataFrame
df = pd.DataFrame(rows)
# clan_monthly_performance_JUL_2024_to_MAY_2025.csv
df.to_csv("clan_monthly_performance.csv", index=False)
print("Saved as clan_monthly_performance.csv")

Processing JUL-AUG_2024...
Processing AUG-SEP_2024...
Processing SEP-OCT_2024...
Processing OCT-NOV_2024...
Processing NOV-DEC_2024...
Processing DEC-JAN_2025...
Processing JAN-FEB_2025...
Processing FEB-MAR_2025...
Processing MAR-APR_2025...
Processing APR-MAY_2025...
Saved as clan_monthly_performance.csv


In [ ]:
import pandas as pd

# URL of the CSV file
csv_file = "clan_monthly_performance.csv"

# Read the CSV data
df = pd.read_csv(csv_file)

# Convert DataFrame to JSON format
json_data = df.to_json(orient='records', indent=2)

# Save to a JSON file
json_filename = "clan_monthly_performance.json"
with open(json_filename, "w") as json_file:
    json_file.write(json_data)

print(f"Saved JSON file as: {json_filename}")

Saved JSON file as: clan_monthly_performance.json
